In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

In [9]:
df_customer = pd.read_csv("/Users/minji/fastprojects/open/Customer_info.csv")
df_discount = pd.read_csv("/Users/minji/fastprojects/open/Discount_info.csv")
df_marketing = pd.read_csv("/Users/minji/fastprojects/open/Marketing_info.csv")
df_sales = pd.read_csv("/Users/minji/fastprojects/open/Onlinesales_info.csv")
df_tax = pd.read_csv("/Users/minji/fastprojects/open/Tax_info.csv")


In [11]:
# [2] 날짜 타입 변환 + 월 추출
df_sales['거래날짜'] = pd.to_datetime(df_sales['거래날짜'])
df_sales['월'] = df_sales['거래날짜'].dt.strftime('%b')  # Jan, Feb 형식

df_marketing['날짜'] = pd.to_datetime(df_marketing['날짜'])  # 나중에 날짜 기준 join 가능

In [12]:
# [3] 고객 정보 merge (고객ID 기준)
df_merged = df_sales.merge(df_customer, on='고객ID', how='left')

In [13]:
# [4] 세금 정보 merge (제품카테고리 기준)
df_merged = df_merged.merge(df_tax, on='제품카테고리', how='left')

In [14]:
# [5] 할인율 정보 merge (제품카테고리 + 월 기준)
df_merged = df_merged.merge(df_discount[['제품카테고리', '월', '쿠폰코드', '할인율']], 
                            on=['제품카테고리', '월'], how='left')

In [15]:
# [6] 거래금액 계산
df_merged['거래금액'] = df_merged['수량'] * df_merged['평균금액'] * (1 + df_merged['GST']) * (1 - df_merged['할인율']/100)

In [16]:
# [7] (선택) 마케팅비용 merge (날짜 기준)
df_merged = df_merged.merge(df_marketing, left_on='거래날짜', right_on='날짜', how='left')


In [17]:
# [8] (선택) 날짜 컬럼 정리
df_merged.drop(columns=['날짜'], inplace=True, errors='ignore')  # 중복 제거용

In [18]:
# [9] 결과 확인
print(df_merged.shape)
df_merged.head()

(52924, 19)


,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,월,성별,고객지역,가입기간,GST,쿠폰코드,할인율,거래금액,오프라인비용,온라인비용
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,Jan,남,Chicago,12,0.10,ELEC10,10.0,152.1729,4500,2424.5
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,Jan,남,Chicago,12,0.10,ELEC10,10.0,152.1729,4500,2424.5
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.5,Used,Jan,남,Chicago,12,0.10,OFF10,10.0,2.0295,4500,2424.5
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.5,Not Used,Jan,남,Chicago,12,0.18,SALE10,10.0,93.0843,4500,2424.5
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.5,Used,Jan,남,Chicago,12,0.18,AIO10,10.0,17.5230,4500,2424.5


In [19]:
df_merged.shape

(52924, 19)

In [20]:
df_merged.columns

Index(['고객ID', '거래ID', '거래날짜', '제품ID', '제품카테고리', '수량', '평균금액', '배송료', '쿠폰상태',
       '월', '성별', '고객지역', '가입기간', 'GST', '쿠폰코드', '할인율', '거래금액', '오프라인비용',
       '온라인비용'],
      dtype='object')

In [21]:
df_merged.dtypes

고객ID              object
거래ID              object
거래날짜      datetime64[ns]
제품ID              object
제품카테고리            object
수량                 int64
평균금액             float64
배송료              float64
쿠폰상태              object
월                 object
성별                object
고객지역              object
가입기간               int64
GST              float64
쿠폰코드              object
할인율              float64
거래금액             float64
오프라인비용             int64
온라인비용            float64
dtype: object

In [22]:
df_merged.isnull().sum()

고객ID        0
거래ID        0
거래날짜        0
제품ID        0
제품카테고리      0
수량          0
평균금액        0
배송료         0
쿠폰상태        0
월           0
성별          0
고객지역        0
가입기간        0
GST         0
쿠폰코드      400
할인율       400
거래금액      400
오프라인비용      0
온라인비용       0
dtype: int64

In [23]:
# 결측치가 1개 이상 있는 컬럼만 출력
df_merged.isnull().sum()[df_merged.isnull().sum() > 0]


쿠폰코드    400
할인율     400
거래금액    400
dtype: int64

In [24]:
# 할인율이 결측치인 row만 확인
df_merged[df_merged['할인율'].isnull()][['고객ID', '제품카테고리', '월', '쿠폰코드', '할인율', '거래금액']].head(10)


,고객ID,제품카테고리,월,쿠폰코드,할인율,거래금액
62,USER_1358,Fun,Jan,NaN,NaN,NaN
95,USER_0585,Backpacks,Jan,NaN,NaN,NaN
157,USER_1421,Fun,Jan,NaN,NaN,NaN
178,USER_0906,Fun,Jan,NaN,NaN,NaN
193,USER_0954,Google,Jan,NaN,NaN,NaN
314,USER_0284,Google,Jan,NaN,NaN,NaN
439,USER_0484,More Bags,Jan,NaN,NaN,NaN
525,USER_1378,More Bags,Jan,NaN,NaN,NaN
573,USER_1382,Google,Jan,NaN,NaN,NaN
648,USER_0138,Google,Jan,NaN,NaN,NaN


In [25]:
# 결측치 있는 열만 가져오기
null_columns = df_merged.columns[df_merged.isnull().any()]

# 각 열에 대해 결측치인 행 일부 확인
for col in null_columns:
    print(f"\n🔍 [결측치 있는 열: {col}]\n")
    display(df_merged[df_merged[col].isnull()][[col, '고객ID', '제품카테고리', '월', '거래날짜']].head())



🔍 [결측치 있는 열: 쿠폰코드]



,쿠폰코드,고객ID,제품카테고리,월,거래날짜
62,NaN,USER_1358,Fun,Jan,2019-01-01
95,NaN,USER_0585,Backpacks,Jan,2019-01-02
157,NaN,USER_1421,Fun,Jan,2019-01-02
178,NaN,USER_0906,Fun,Jan,2019-01-02
193,NaN,USER_0954,Google,Jan,2019-01-02



🔍 [결측치 있는 열: 할인율]



,할인율,고객ID,제품카테고리,월,거래날짜
62,NaN,USER_1358,Fun,Jan,2019-01-01
95,NaN,USER_0585,Backpacks,Jan,2019-01-02
157,NaN,USER_1421,Fun,Jan,2019-01-02
178,NaN,USER_0906,Fun,Jan,2019-01-02
193,NaN,USER_0954,Google,Jan,2019-01-02



🔍 [결측치 있는 열: 거래금액]



,거래금액,고객ID,제품카테고리,월,거래날짜
62,NaN,USER_1358,Fun,Jan,2019-01-01
95,NaN,USER_0585,Backpacks,Jan,2019-01-02
157,NaN,USER_1421,Fun,Jan,2019-01-02
178,NaN,USER_0906,Fun,Jan,2019-01-02
193,NaN,USER_0954,Google,Jan,2019-01-02


In [26]:
# 할인율 없으면 0% 할인으로 간주
df_merged['할인율'] = df_merged['할인율'].fillna(0)

In [27]:
# 쿠폰코드 없으면 'NoCoupon'으로 채우기
df_merged['쿠폰코드'] = df_merged['쿠폰코드'].fillna('NoCoupon')


In [28]:
# 거래금액 다시 계산 (할인율 포함)
df_merged['거래금액'] = (
    df_merged['수량'] * 
    df_merged['평균금액'] * 
    (1 + df_merged['GST']) * 
    (1 - df_merged['할인율'] / 100)
)

In [29]:
df_merged.isnull().sum()

고객ID      0
거래ID      0
거래날짜      0
제품ID      0
제품카테고리    0
수량        0
평균금액      0
배송료       0
쿠폰상태      0
월         0
성별        0
고객지역      0
가입기간      0
GST       0
쿠폰코드      0
할인율       0
거래금액      0
오프라인비용    0
온라인비용     0
dtype: int64

In [30]:
df_merged.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 52924 entries, 0 to 52923
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   고객ID    52924 non-null  object        
 1   거래ID    52924 non-null  object        
 2   거래날짜    52924 non-null  datetime64[ns]
 3   제품ID    52924 non-null  object        
 4   제품카테고리  52924 non-null  object        
 5   수량      52924 non-null  int64         
 6   평균금액    52924 non-null  float64       
 7   배송료     52924 non-null  float64       
 8   쿠폰상태    52924 non-null  object        
 9   월       52924 non-null  object        
 10  성별      52924 non-null  object        
 11  고객지역    52924 non-null  object        
 12  가입기간    52924 non-null  int64         
 13  GST     52924 non-null  float64       
 14  쿠폰코드    52924 non-null  object        
 15  할인율     52924 non-null  float64       
 16  거래금액    52924 non-null  float64       
 17  오프라인비용  52924 non-null  int64         
 18  온라인비용 

In [31]:
월_변환 = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
          'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
          'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

df_merged['월(숫자형)'] = df_merged['월'].map(월_변환)


In [32]:
df_merged.describe()


,수량,평균금액,배송료,가입기간,GST,할인율,거래금액,오프라인비용,온라인비용,월(숫자형)
count,52924.000000,52924.000000,52924.000000,52924.000000,52924.000000,52924.000000,52924.000000,52924.000000,52924.000000,52924.000000
mean,4.497638,52.237646,10.517630,26.127995,0.137462,19.802358,78.563156,2830.914141,1893.109119,6.652388
std,20.104711,64.006882,19.475613,13.478285,0.045825,8.278878,149.875728,936.154247,807.014092,3.333364
min,1.000000,0.390000,0.000000,2.000000,0.050000,0.000000,0.315700,500.000000,320.250000,1.000000
25%,1.000000,5.700000,6.000000,15.000000,0.100000,10.000000,11.225340,2500.000000,1252.630000,4.000000
50%,1.000000,16.990000,6.000000,27.000000,0.180000,20.000000,28.865160,3000.000000,1837.870000,7.000000
75%,2.000000,102.130000,6.500000,37.000000,0.180000,30.000000,114.730000,3500.000000,2425.350000,9.000000
max,900.000000,355.740000,521.360000,50.000000,0.180000,30.000000,8972.775000,5000.000000,4556.930000,12.000000


In [37]:
df_merged['제품카테고리'].unique()


array(['Nest-USA', 'Office', 'Apparel', 'Bags', 'Drinkware', 'Lifestyle',
       'Notebooks & Journals', 'Headgear', 'Waze', 'Fun', 'Nest-Canada',
       'Backpacks', 'Google', 'Bottles', 'Gift Cards', 'More Bags',
       'Housewares', 'Android', 'Accessories', 'Nest'], dtype=object)

In [38]:
df_merged.to_csv('/Users/minji/fastprojects/open/merge_data.csv', index=False)
